# Centrum en spreiding

In [ ]:
import pandas as pd
import numpy as np
import math
from scipy import stats
import matplotlib.pyplot as plt

In [ ]:
laptops = pd.read_csv('laptops.csv', sep=";", decimal=",", header=0)
cpu_generation_levels = [ "Sandy Bridge", "Ivy Bridge", "Haswell", "Broadwell", "Skylake", "Kabylake" ]
laptops.cpuGeneration = pd.Categorical(laptops.cpuGeneration, ordered=True, categories=cpu_generation_levels)
cpu_type_levels = [ "i3", "i5", "i7" ]
laptops.cpuType = pd.Categorical(laptops.cpuType, ordered=True, categories=cpu_type_levels)
laptops.brand = pd.Categorical(laptops.brand)

## Modus

Wat is een modus???
Vanaf welk meetniveau?

In [ ]:
laptops.cpuGeneration.mode() # dit levert een Series op (kunnen er meerdere waarden zijn?)

(pseudo) continue variabelen:

In [ ]:
cutpoints = [ 0, 120, 250, 500, 1000 ]
klassen = pd.cut(laptops.diskspace, bins=cutpoints)
klassen.mode()

opmerkingen:
- afhankelijk van de opdeling in waarden
- er kan meer dan 1 modus zijn
- de waarde van de modus is altijd een bestaande waarde
- de modus verandert niet als er uitschieters zijn

## Mediaan

Wat is een mediaan?
Vanaf welk meetniveau?

In [ ]:
laptops.RAM.median()

In [ ]:
# waarom geeft het volgende een fout?
# laptops.cpuGeneration.median()

In [ ]:
def median_categorical(data):
   d = data.dropna()
   n = len(d)
   middle = math.floor(n/2)
   return d.sort_values().iloc[middle]

In [ ]:
laptops.RAM.median()

In [ ]:
median_categorical(laptops.cpuGeneration)

In [ ]:
median_categorical(laptops.brand) # heeft dit zin?

In [ ]:
# alternatief
print(laptops.cpuGeneration.quantile(0.5))
print(laptops.brand.quantile(0.5)) # zinvol?

opmerkingen:
- ongevoelig voor uitschieters
- je hoeft geen klassen te maken bij continue data
- de mediaan is altijd een bestaand getal (tenzij je een even aantal had)
- de andere waarden hebben geen invloed
- sorteren kost tijd
- er is geen wiskundige formule voor de mediaan

## Gemiddelde

Wat is een gemiddelde?
Vanaf welk meetniveau?

In [ ]:
laptops.diskspace.mean().round(1)

Gemiddelde vanuit een frequentietabel?

In [ ]:
freq = laptops.RAM.value_counts().sort_index()
print(freq)

In [ ]:
waarden = freq.index
(freq*waarden).sum()/freq.sum()

In [ ]:
laptops.RAM.mean() # verifiëren of dit overeenkomt met het vorige

opmerkingen:
- er is een wiskundige formule: handiger om eigenschappen te bewijzen
- alle waarden spelen een rol in het resultaat
- het resultaat is niet noodzakelijk een bestaande waarde
- uitschieters spelen een zeer grote rol

## Andere gemiddelden

gewogen gemiddelde

In [ ]:
# voorbeeld:
vak = ['Computersystemen 1', 'OO programmeren 1', 'Datastr. en algoritmes', 'Data-Science 1', 'Netwerkarchitectuur 1', 'Communicatie 1', 'User interfaces 1', 'Databanken 1', 'Software engineering 1', 'Boekhouden', 'Management accounting']
score = pd.Series([18,15,12,10,18,13,17,15,13,12,12])
sp = pd.Series([6,11,5,3,3,7,4,7,7,3,4])
print(pd.DataFrame({'vak':vak, 'score':score, 'studiepunten':sp}))

In [ ]:
(sp * score).sum()/sp.sum()

meetkundig gemiddelde

In [ ]:
koers = pd.Series([ 5, 3, -2 ])
koers = koers/100 + 1
(np.exp(np.mean(np.log(koers)))-1)*100

In [ ]:
(stats.gmean(koers)-1)*100

harmonisch gemiddelde

In [ ]:
snelheid = pd.Series([ 120, 100 ])
print(1 / np.mean(1/snelheid))
print(stats.hmean(snelheid))

voortschrijdend gemiddelde: zie later

## Spreiding

In [ ]:
x1 = pd.Series([1, 2, 3, 5, 46, 87, 88, 89, 90])
x2 = pd.Series([42, 43, 44, 45, 46, 47, 47, 48, 49])

In [ ]:
print(x1.mean())
print(x2.mean())
print(x1.median())
print(x2.median())

# Bereik

In [ ]:
print(x1.max()-x1.min())
print(x2.max()-x2.min())

## Interkwartielafstand

'kwartielen': veralgemening van mediaan

In [ ]:
print(laptops.diskspace.quantile(q=[0.25,0.5,0.75]))

decielen

In [ ]:
print(laptops.diskspace.quantile(q=np.arange(0.1, 1, 0.1).tolist()))

interkwartielafstand = Q3 - Q1

In [ ]:
stats.iqr(laptops.diskspace.dropna())

Boxplot:

In [ ]:
fig, ax = plt.subplots()
ax.boxplot(laptops.diskspace.dropna())
ax.set_title('Schijfruimte in laptops')
ax.set_ylabel('Schijfruimte (in GB)')
# fig.show()

Wat is dat donkere gedeelte bovenaan?

## Standaardafwijking

In [ ]:
x = laptops.diskspace.dropna()
verschillen = x - x.mean()
verschillen.mean()

In [ ]:
verschillen = (x - x.mean()).abs()
verschillen.mean()

In [ ]:
x.mad() # deprecated...

In [ ]:
verschillen = (x - x.mean())**2
math.sqrt(verschillen.mean())

In [ ]:
x.std() # dit is lichtjes verschillend...

"Bessel" correctie:

In [ ]:
n = len(x)
verschillen = (x - x.mean())**2
math.sqrt(verschillen.mean()*n/(n-1))

## Uitschieters

invloed van uitschieters:

In [ ]:
x = laptops.RAM.copy()
print(x.mean())
print(x.std())
print(x.median())
print(stats.iqr(x.dropna()))

In [ ]:
x[42] = 8192
print(x.mean())
print(x.std())
print(x.median())
print(stats.iqr(x.dropna()))

Grenzen voor uitschieters vinden:

In [ ]:
def outlier_boundaries(x, factor=1.5):
   Q1 = x.quantile(0.25)
   Q3 = x.quantile(0.75)
   I = Q3 - Q1
   return [ Q1 - factor * I, Q3 + factor * I ]

In [ ]:
x = laptops.RAM.copy()
low, high = outlier_boundaries(x)
print(low)
print(high)

In [ ]:
geen_uitschieters = x[x.between(low, high)]
print(geen_uitschieters)

In [ ]:
uitschieters = x[~x.between(low, high)]
print(uitschieters)

In [ ]:
print(list(uitschieters.index)) # rijnummers met uitschieters

## Z-scores

Wat gebeurt er met het gemiddelde en standaardafwijking als je:
- alle waarden verminderd met 2
- alle waarden vermeerderd met 2
- alle waarden vermenigvuldigd met 5
- alle waarden deelt door 5

In [ ]:
# met de hand:
x = laptops.diskspace.dropna()
s = x.std()
z = (x - x.mean())/s
print(z.head())

In [ ]:
print(z.mean())
print(z.std())

In [ ]:
z = stats.zscore(laptops.diskspace.dropna(), ddof=1)
print(z.head())

In [ ]:
print(z.mean())
print(z.std())

In [ ]:
fig, ax = plt.subplots()
ax.boxplot([laptops.diskspace.dropna(), laptops.RAM.dropna()], labels=['schijfruimte', 'RAM'])
ax.set_title('Opslagruimte')
ax.set_ylabel('in GB')
# fig.show()

In [ ]:
diskpace_z = stats.zscore(laptops.diskspace.dropna(), ddof=1)
ram_z = stats.zscore(laptops.RAM.dropna(), ddof=1)
fig, ax = plt.subplots()
ax.boxplot([diskpace_z, ram_z], labels=['schijfruimte', 'RAM'])
ax.set_title('Opslagruimte')
ax.set_ylabel('Z-score')
# fig.show()